In [17]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from transformers import BitsAndBytesConfig
from datasets import load_from_disk
import torch
import os
import traceback
from pathlib import Path


In [2]:
def load_model_and_tokenizer(model_name: str, local_dir: str = "./model/TinyLlama"):

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
    )

    if os.path.exists(local_dir) and os.path.isdir(local_dir):
        print(f"🔁 Загрузка модели из локального каталога: {local_dir}")
        tokenizer = AutoTokenizer.from_pretrained(local_dir)
        model = AutoModelForCausalLM.from_pretrained(
            local_dir,
            device_map="auto",
            torch_dtype=torch.float16,
            quantization_config=bnb_config
        )
    else:
        print(f"⬇️ Модель не найдена локально, загружаем из интернета и сохраняем в: {local_dir}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16,
            quantization_config=bnb_config
        )
        # Сохраняем модель и токенизатор в локальный каталог
        tokenizer.save_pretrained(local_dir)
        model.save_pretrained(local_dir)

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    model.config.use_cache = False
    model.config.pretraining_tp = 1

    return tokenizer, model


In [3]:
def apply_lora(model):
    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, lora_config)
    model.gradient_checkpointing_enable()  

    return model

In [4]:
def tokenize_russian_dialogues(batch, tokenizer, max_length=512):
    texts = []

    for question, answer in zip(batch["question"], batch["answer"]):
        if question is None or answer is None:
            continue  # пропускаем пустые

        prompt = question.strip()
        response = answer.strip()

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )

In [5]:
def tokenize_gsm8k_and_MAWPS(batch, tokenizer, max_length=512):
    texts = []

    for question, answer in zip(batch["question"], batch["answer"]):
        if question is None or answer is None:
            continue

        prompt = question.strip()
        response = answer.strip()

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )

In [6]:
def tokenize_asdiv(batch, tokenizer, max_length=512):
    texts = []

    for question, answer in zip(batch["text"], batch["target"]):
        if question is None or answer is None:
            continue

        prompt = question.strip()
        response = answer.strip()

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )


In [7]:
def tokenize_mathqa(batch, tokenizer, max_length=512):
    texts = []

    for question, chain, result in zip(batch["question"], batch["chain"], batch["result"]):
        if question is None or chain is None or result is None:
            continue

        prompt = question.strip()
        response = f"{chain.strip()} → {result.strip()}"

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )


In [8]:
def tokenize_openbookqa(batch, tokenizer, max_length=512):
    texts = []

    for stem, choices, answer_key in zip(batch["question_stem"], batch["choices"], batch["answerKey"]):
        if stem is None or choices is None or answer_key is None:
            continue

        prompt = stem.strip()

        # выбор правильного варианта по answerKey
        label_list = choices["label"]
        text_list = choices["text"]
        label_to_text = {label: text for label, text in zip(label_list, text_list)}
        response = label_to_text.get(answer_key, "").strip()

        if not response:
            continue

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )


In [9]:
def tokenize_dataset(dataset, tokenizer, tokenize_fn, max_length=512):
    def wrapped(example):
        return tokenize_fn(example, tokenizer, max_length=max_length)

    return dataset.map(wrapped, batched=True, remove_columns=dataset.column_names)

In [10]:
def train_lora(
    model_name: str,
    dataset_path: str,
    output_dir: str,
    epochs: int = 3,
    batch_size: int = 4,
    lr: float = 2e-4,
    max_length: int = 512,
    tokenize_fn=None
):
    # Загрузка
    tokenizer, model = load_model_and_tokenizer(model_name)
    model = apply_lora(model)
    dataset = load_from_disk(dataset_path)
    dataset = tokenize_dataset(
        dataset,
        tokenizer=tokenizer,
        tokenize_fn=tokenize_fn,
        max_length=max_length
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # обязательно для CausalLM
    )


    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.deterministic = False


    # Аргументы обучения
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        lr_scheduler_type="cosine",
        warmup_steps=100,
        num_train_epochs=epochs,
        learning_rate=lr,
        fp16=True,
        logging_steps=20,
        save_steps=200,
        save_total_limit=2,
        report_to="none"
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator  
    )

    # Обучение
    trainer.train()
    model.save_pretrained(output_dir)

    del model
    del tokenizer
    torch.cuda.empty_cache()

In [11]:
def merge_lora_adapter(
    base_model_path: str,
    adapter_path: str,
    save_path: str
):
    try:
        # Загружаем базовую модель и токенизатор через универсальную функцию
        print("Загрузка модели и токенизатора")
        tokenizer, model = load_model_and_tokenizer(base_model_path)

        # Загружаем LoRA и объединяем
        print("Объединение модели и адаптера")
        model = PeftModel.from_pretrained(model, adapter_path)
        model = model.merge_and_unload()

        # Сохраняем объединённую модель
        print("Сохранение объединенной модели и токенизатора")
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)

        print(f"✅ Объединённая модель сохранена в: {save_path}")
    except Exception as e:
        print(f"Ошибка: {e}")
        traceback.print_exc()
    try:
        print("Очистка памяти")
        del model
        del tokenizer
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"Ошибка: {e}")
        traceback.print_exc()

In [18]:
# train_lora(
#     model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     dataset_path="./data/russian_dialogues_20k",
#     output_dir="./checkpoints/step_dialogue",
#     epochs=3,
#     batch_size=4,
#     tokenize_fn=tokenize_russian_dialogues
# )
merge_lora_adapter(
    base_model_path="TinyLlama/TinyLlama-1.1B-Chat-v1.0",                 # ← локально сохранённая/обновлённая модель
    adapter_path=Path("checkpoints/step_dialogue/checkpoint-8200").resolve(),           # ← путь к обученному адаптеру
    save_path="./model/TinyLlama"                                         # ← перезаписываем модель
)

Загрузка модели и токенизатора
⬇️ Модель не найдена локально, загружаем из интернета и сохраняем в: ./model/TinyLlama
Объединение модели и адаптера


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Сохранение объединенной модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
Очистка памяти


In [19]:
train_lora(
    model_name="./model/TinyLlama",                            # ← обновлённая модель после 1-го этапа
    dataset_path="./data/gsm8k_sel",                           # ← путь к сохранённому датасету
    output_dir="./checkpoints/step_math",                      # ← новый чекпоинт для math
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_gsm8k_and_MAWPS                       # ← твоя функция токенизации
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                       # ← обновлённая после step_dialogue
    adapter_path=Path("./checkpoints/step_math").resolve(),    # ← LoRA второго этапа
    save_path="./model/TinyLlama"                              # ← перезаписываем итоговую модель
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama


Map: 100%|██████████| 5000/5000 [00:01<00:00, 3313.23 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_24572\510607462.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.610000
40,1.416100
60,1.149100
80,1.014400
100,0.960000
120,0.928900
140,0.955600
160,0.955200
180,0.936200
200,0.923600


Загрузка модели и токенизатора
🔁 Загрузка модели из локального каталога: ./model/TinyLlama


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\quantizers\auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Объединение модели и адаптера


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Сохранение объединенной модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
Очистка памяти


In [20]:
train_lora(
    model_name="./model/TinyLlama",                            
    dataset_path="./data/mawps_sel",                          
    output_dir="./checkpoints/step_mawps",                     
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_gsm8k_and_MAWPS                     
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                      
    adapter_path=Path("./checkpoints/step_mawps").resolve(),             
    save_path="./model/TinyLlama"                              
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama


Map: 100%|██████████| 1417/1417 [00:00<00:00, 3449.97 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_24572\510607462.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.670700
40,1.428800
60,1.050100
80,0.955500
100,0.923800
120,0.865700
140,0.849900
160,0.841100
180,0.818200
200,0.763200


Загрузка модели и токенизатора
🔁 Загрузка модели из локального каталога: ./model/TinyLlama


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\quantizers\auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Объединение модели и адаптера


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Сохранение объединенной модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
Очистка памяти


In [21]:
train_lora(
    model_name="./model/TinyLlama",                            
    dataset_path="./data/asdiv_sel",                          
    output_dir="./checkpoints/step_asdiv",                     
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_asdiv                     
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                      
    adapter_path=Path("./checkpoints/step_asdiv").resolve(),       
    save_path="./model/TinyLlama"                              
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama


Map: 100%|██████████| 1200/1200 [00:00<00:00, 3521.46 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_24572\510607462.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,2.012600
40,1.544000
60,1.132800
80,1.030700
100,1.036400
120,0.974200
140,1.011700
160,0.946700
180,0.949500
200,0.969900


Загрузка модели и токенизатора
🔁 Загрузка модели из локального каталога: ./model/TinyLlama


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\quantizers\auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Объединение модели и адаптера


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Сохранение объединенной модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
Очистка памяти


In [22]:
train_lora(
    model_name="./model/TinyLlama",                            
    dataset_path="./data/mathqa_sel",                          
    output_dir="./checkpoints/step_mathqa",                     
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_mathqa                     
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                      
    adapter_path=Path("./checkpoints/step_mathqa").resolve(),             
    save_path="./model/TinyLlama"                              
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama


Map: 100%|██████████| 5000/5000 [00:01<00:00, 3349.21 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_24572\510607462.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.385900
40,1.113500
60,0.813300
80,0.705000
100,0.653300
120,0.625700
140,0.615200
160,0.608900
180,0.594800
200,0.596100


Загрузка модели и токенизатора
🔁 Загрузка модели из локального каталога: ./model/TinyLlama


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\quantizers\auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Объединение модели и адаптера


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Сохранение объединенной модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
Очистка памяти


In [23]:
train_lora(
    model_name="./model/TinyLlama",                            
    dataset_path="./data/openbookqa_sel",                          
    output_dir="./checkpoints/step_openbookqa",                     
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_openbookqa                     
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                      
    adapter_path=Path("./checkpoints/step_openbookqa").resolve(),       
    save_path="./model/TinyLlama"                              
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama


Map: 100%|██████████| 4957/4957 [00:01<00:00, 3525.48 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_24572\510607462.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.595000
40,1.443700
60,1.316400
80,1.260900
100,1.243000
120,1.231500
140,1.218900
160,1.230400
180,1.228000
200,1.206500


Загрузка модели и токенизатора
🔁 Загрузка модели из локального каталога: ./model/TinyLlama


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\quantizers\auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Объединение модели и адаптера


c:\Users\Настя\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\tuners\lora\bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Сохранение объединенной модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
Очистка памяти
